# 创建训练流水线（容器版本）

## 导语

此笔记旨在创建一个训练流水线，以便我们可以训练一个模型。

In [1]:
HOST = "https://aip-apis.nadileaf.com"
PROJECT_ID = "tokyo-rain-42"
LOCATION = "default"

## 发起「创建训练流水线」请求

我们使用 `IMAGE_URL` 作为容器映像的 URL，并执行 `COMMAND` 命令以及 `ARGS` 参数。

In [2]:
from datetime import datetime

MACHINE_TYPE = "TBD"
ACCELERATOR_TYPE = "ACCELERATOR_TYPE_UNSPECIFIED"
ACCELERATOR_COUNT = 0  # number of GPUs

IMAGE_URL = "harbor.nadileaf.com/aip-images/create-training-image:0.1.1"
COMMAND = ["echo"]
ARGS = ["hello world"]

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DISPLAY_NAME = "create-train-pipeline_" + TIMESTAMP

DISPLAY_NAME

'create-train-pipeline_20220819165116'

In [3]:
from devtools import pprint
import requests
import json

url = f"{HOST}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/trainingPipelines"

payload = json.dumps(
    {
        "displayName": DISPLAY_NAME,
        "trainingTaskDefinition": "s3://aip/schema/trainingjob/definition/custom_task_1.0.0.yaml",
        "trainingTaskInputs": {
            "workerPoolSpecs": [
                {
                    "machineSpec": {
                        "machineType": MACHINE_TYPE,
                        "acceleratorType": ACCELERATOR_TYPE,
                        "acceleratorCount": ACCELERATOR_COUNT,
                    },
                    "containerSpec": {
                        "imageUri": IMAGE_URL,
                        "command": COMMAND,
                        "args": ARGS,
                    },
                }
            ]
        },
        "modelToUpload": {
            "displayName": DISPLAY_NAME,
            "predictSchemata": {},
            "containerSpec": {
                "imageUri": "tensorflow/serving:latest",
                "ports": [{"containerPort": 8501}],
                "env": [
                    {"name": "MODEL_BASE_PATH", "value": "/app"},
                    {"name": "MODEL_NAME", "value": "naive_model"},
                ],
                "predictRoute": "/v1/models/naive_model:predict",
            },
        },
    }
)
headers = {"Content-Type": "application/json"}

response = requests.request("POST", url, headers=headers, data=payload)

pprint(response.json())

{
    'name': 'projects/tokyo-rain-42/locations/default/trainingPipelines/30c0238d-4b89-492c-af26-07aae16c31ce',
    'displayName': 'create-train-pipeline_20220819165116',
    'trainingTaskDefinition': 's3://aip/schema/trainingjob/definition/custom_task_1.0.0.yaml',
    'trainingTaskInputs': {
        'workerPoolSpecs': [
            {
                'machineSpec': {
                    'machineType': 'TBD',
                    'acceleratorType': 'ACCELERATOR_TYPE_UNSPECIFIED',
                    'acceleratorCount': 0,
                },
                'containerSpec': {
                    'imageUri': 'harbor.nadileaf.com/aip-images/create-training-image:0.1.1',
                    'command': ['echo'],
                    'args': [
                        'hello world',
                    ],
                },
            },
        ],
    },
    'modelToUpload': {
        'name': '',
        'displayName': 'create-train-pipeline_20220819165116',
        'description': '',
       

## 获取「训练流水线」的「名字」

In [4]:
TRAINING_PIPELINE_NAME = response.json()["name"]
TRAINING_PIPELINE_NAME

'projects/tokyo-rain-42/locations/default/trainingPipelines/30c0238d-4b89-492c-af26-07aae16c31ce'

## 查询「训练流水线」的「状态」

直到 `state` 字段为 `PIPELINE_STATE_SUCCEEDED`，才能获取到模型的名字。

In [7]:
import requests
from devtools import pprint

url = f"{HOST}/v1/{TRAINING_PIPELINE_NAME}"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

pprint(response.json()["state"])
pprint(response.json())

'PIPELINE_STATE_SUCCEEDED'
{
    'name': 'projects/tokyo-rain-42/locations/default/trainingPipelines/30c0238d-4b89-492c-af26-07aae16c31ce',
    'displayName': 'create-train-pipeline_20220819165116',
    'trainingTaskDefinition': 's3://aip/schema/trainingjob/definition/custom_task_1.0.0.yaml',
    'trainingTaskInputs': {
        'workerPoolSpecs': [
            {
                'machineSpec': {
                    'machineType': 'TBD',
                    'acceleratorType': 'ACCELERATOR_TYPE_UNSPECIFIED',
                    'acceleratorCount': 0,
                },
                'containerSpec': {
                    'args': [
                        'hello world',
                    ],
                    'command': ['echo'],
                    'imageUri': 'harbor.nadileaf.com/aip-images/create-training-image:0.1.1',
                },
            },
        ],
    },
    'modelToUpload': {
        'name': 'projects/tokyo-rain-42/locations/default/models/926df648-415d-47c1-8e81-c

## 获取「模型」的「名字」

In [52]:
MODEL_NAME = response.json()["modelToUpload"]["name"]
MODEL_NAME

'projects/tokyo-rain-42/locations/default/models/750e6e67-526c-44a3-b173-05ea95a9cc43'